# Survival C++ and how to use it to work with data

In order to understand the course material we must first understand the C language and some basic concepts from C++. Although C++ is the language of choice for many real-world scientific applications, it is more complex to understand than plain C, and can potentially introduce uncecessary complexity that may get in the way of effectively grasping the course material. This course will use mostly the C subset of the C++ language, however we use C++ compilers and tools so that we can use more advanced C++ features if convenient.

## Comments

Comments in C++ begin with two forward slashes //. Anything after the two forward slashes is ignored for the rest of the line.

```C++
// This is a comment
```

## Statements and code blocks

A statement is a line of code. Since C++ doesn't use new lines to separate statements then every statement must have an ending, C++ uses the semicolon ";" character to end statements.

```C++
// This is a statement that declares and integer with value 2
int a = 2;
```

**Code blocks** are collections of statements enclosed by a starting brace "{", and ending with an end brace "}". Variables declared within code blocks do not exist outside their enclosing code block.

## Basic data types

There are only really two kinds of data types, integers and floats, with varying numbers of bits to represent them. Every other data type is a reinterpretation or compound mixture of these basic data types.





* Basic data types
* OpenCL Data types
* Stacks and heaps, allocating and de-allocating memory
* Reading and writing to and from a file
* Structures
* Multi-dimensional array access
* Functions
* math functions
* programs
* Exercise: debugging with GDB